Notebook to backtest with backtrader the ML scores built for smart DCA on CW8

In [ ]:
from importlib import reload
import pandas as pd
import backtrader as bt
import bt_ml_strategies as strat
reload(strat)

PATH_DATA_BT = "../../Data/backtest/"



In [ ]:
df = pd.read_csv(PATH_DATA_BT+"DCA_CLOSE_1D_V1_lab_perf_21d_predict_BT.zip", sep=",",
                            index_col="OPEN_DATETIME", parse_dates=True, keep_default_na=True, low_memory=False)

In [ ]:
prices=df[['open','high','low','close','volume','Predict','SL','TP']]
prices[['SL','TP']]=0
prices=prices['2010-04-01 00:00:00':]
prices.dropna(inplace=True)
prices.sort_index(inplace=True)
prices.info()

In [ ]:
data = strat.SignalData(dataname=prices,timeframe=bt.TimeFrame.Days)#, compression=30)

In [ ]:
dca_params = {'score_buy': 0.00, 'score_sell': -10.005, 'size_buy': 2500, 'size_sell': 2500}
cerebro = bt.Cerebro(cheat_on_open=True)
cerebro.addstrategy(strat.BtMlDcaStrategy,**dca_params)
cerebro.adddata(data)
cerebro.broker.setcash(1000000.0)
cerebro.broker.setcommission(commission=0.0025)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")

print(f"Starting portfolio:{cerebro.broker.getvalue()}")
backtest_result=cerebro.run()
print(f"Ending portfolio:{cerebro.broker.getvalue()}")